# Processamento Plano Funding

## Objtetivo

<li> Criar plano de funding a ser utilizado nos casamentos de origens e aplicações
<li> Parametrizar o plano para conciliar saldos de produtos com saldos contábeis
<li> Parametrizar o plano para permitir classificar o passivo/ativo quanto a sensibilidade de juros e prazos
<li> Parametrizar o plano de acordo com o custo de oportunidade a ser utilizado para rentabilizar recursos captados e precificar recursos aplicados

## Import Library

In [1]:
# bibliotecas de manipulação
import pandas as pd
import numpy as np

# bibliotecas de visualização
import matplotlib as plt
import seaborn as sns

# bibliotecas de machine learning

# outras bibliotecas
import datetime
import os
import warnings

In [2]:
# opções para o pandas
pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.mode.use_inf_as_na=True
pd.options.display.max_colwidth=None

# opções para visualizações
%matplotlib inline

# opções para Jupyter notebook
warnings.filterwarnings('ignore')

## Importação de bibliotecas ou classes locais

## Definição de Parâmetros

In [3]:
path_output_plano='../data/output/plano/{}'

## Importação de Dados

<li> Fonte A = Capturado do datamart de Carteira de crédito
<li> Fonte B = Capturado do balanço contábil
<li> Fonte C = Capturado do datamart de captação remunerada
<li> Fonte D = Capturado do datamart de conta corrente
<li> Fonte P = Capturado do datamart de conta capital

In [4]:
# importando dados
path='../data/input/plano/plano-funding-pad.xlsx' # insira aqui o caminho completo do arquivo

df_plano=pd.read_excel(path, sheet_name='PlanoFunding', header=0)
df_prazo=pd.read_excel(path, sheet_name='TabelaPrazosMoedaOrigem', header=0)

#df_plano.drop(columns='NomePrazo', inplace=True)

## Pré Processamento

In [5]:
# dataframe do plano funding definitivo
df=pd.DataFrame()

# contas funding atribuídas com base nos dados de balanço
df_plano_balanco=df_plano[(df_plano.Fonte=='B') | (df_plano.BolDesdobrarConta==1)]

# contas funding atribuídas com base nos dados gerenciais do cognos
df_plano_outros=df_plano[(df_plano.Fonte!='B') & (df_plano.BolDesdobrarConta!=1)]

# adicionando contas mapeadas pela fonte "B" - Balanço - ao dataframe definitivo
df=df.append(df_plano_balanco)


# listando códigos que serão utilizados na construção do plano de funding com base no prazo e moedas
# esses itens serão usados para mapear o balanço a partir do produto, encaixando as operações de acordo com suas características nessas codificações
list_prazos=(
    df_prazo[
        (df_prazo.Origem!='MCD')
         ].CodMoeda.to_list()
)



# adicionando as contas funding mapeadas com origem no cognos ao dataframe definitivo
for i in df_plano_outros.index.to_list():
    t=df_plano_outros.loc[i:i]
    for cod_prazo in list_prazos:
        t.CodMoeda=cod_prazo
        df=df.append(t)

## Merge com Prazos

In [6]:
# fazendo merge do plano funding com plano de prazos e moedas
df=df.merge(df_prazo, how='left', left_on=['CodMoeda'], right_on=['CodMoeda'])


# fazendo o atributo de ordem de utilização de prazos e moedas
df['NumOrdemUtilizacaoMoeda']=df.apply(
    lambda x: x.NumOrdemOrigem if x.BolAplicacao==0 else x.NumOrdemAplicacao, axis=1
)

# deletando colunas não mais necessárias
df.drop(columns=['NumOrdemOrigem', 'NumOrdemAplicacao', 'NomePrazo'], inplace=True)
df.sort_values(by=['TipoSaldo', 'NumOrdemUtilizacaoMoeda', 'OrdemUtilizacaoFunding'], inplace=True)
df.reset_index(inplace=True, drop=True)

## Output

In [7]:
# esse é o modelo de plano de funding
# o objetivo é desdobrar todo o balanço em prazos e taxas e moedas para fazer o casamento
# a ordenação de ordem de utilização de moedas e linhas de recursos é para distribuir os recursos captados para as aplicações
# comissionando as captações pelos preços de transferência
# e precificando as aplicações que usaram os recursos vinculados aos mesmos prazos, taxas e moedas

df.to_excel(path_output_plano.format('plano-funding-definitivo.xlsx'), sheet_name='plano', freeze_panes=(1,1))
